In [4]:
import numpy as np
import scipy as sp
import math
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

names = ["KNN", "SVM", "Gaussian_Process",
         "Decision_Tree", "Random_Forest", "Neural_Net", "AdaBoost",
         "Naive_Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(gamma=2, C=1, kernel="linear"),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)]

parameters = {"KNN":{"n_neighbors":50, "n_jobs":-1},
              "SVM":{"C":1.0,"kernel":"linear","degree":1,"probability":True},
              "GP":{"kernel":"linear","n_restarts_optimizer":2, "n_jobs":-1},
              "DT":{"splitter":"best", "max_depth":100},
              "RF":{"n_estimators":10, "max_depth":100 },
            }
# X, y = make_classification(n_samples = 1000, n_features=4, n_redundant=0, n_informative=2,random_state=1, n_clusters_per_class=1)

In [5]:
def sample_hp_Ai(Ai):
    if(Ai is "KNN"):
        parameters["KNN"]["n_neighbors"] = math.ceil(np.random.uniform(100))
    elif(Ai is "SVM"):
        parameters["SVM"]["C"] = float("{0:.2f}".format(abs(np.random.normal(loc=1.0, scale=1.0))))
        parameters["SVM"]["kernel"] = np.random.choice(["linear", "rbf"])
        parameters["SVM"]["degree"] = np.random.randint(1,4)
    elif(Ai is "GP"):
        parameters["GP"]["C"] = float("{0:.2f}".format(abs(np.random.normal(loc=1.0, scale=1.0))))
        parameters["GP"]["n_restarts_optimizer"] = np.random.randint(1,10)
    elif(Ai is "DT"):
        parameters["DT"]["splitter"] = np.random.choice(["best", "random"])
        parameters["DT"]["max_depth"] =np.random.randint(1,100)
    elif(Ai is "RF"):
        parameters["RF"]["n_estimators"] = np.random.randint(1,20)
        parameters["RF"]["max_depth"] = np.random.randint(1,100)

In [16]:
def get_params(A, B, P, N):
    mu = np.random.dirichlet(P)
    Ni = [int(mu_i*N) for mu_i in mu]
    return mu, Ni

In [17]:
get_params(A="SVM", B="RF", P=[100,50,25,10], N=10)

(array([ 0.51271906,  0.25940917,  0.17652167,  0.0513501 ]), [5, 2, 1, 0])

In [7]:
sample_hp_Ai("KNN")
sample_hp_Ai("SVM")
sample_hp_Ai("GP")
sample_hp_Ai("DT")
sample_hp_Ai("RF")
for key in parameters:
    print("{0}\t{1}".format(key,parameters[key]))

KNN	{'n_neighbors': 4, 'n_jobs': -1}
SVM	{'C': 2.37, 'kernel': 'rbf', 'degree': 2, 'probability': True}
GP	{'kernel': 'linear', 'n_restarts_optimizer': 8, 'n_jobs': -1, 'C': 0.99}
DT	{'splitter': 'best', 'max_depth': 24}
RF	{'n_estimators': 9, 'max_depth': 9}
